# GROUP PROJECT

## 1. Data Cleaning and Preprocessing

In [25]:
import pandas as pd
import numpy as np
from datetime import datetime
import statsmodels.api as sm
from statsmodels.tsa.ar_model import AutoReg

In [2]:
ETHUSDT = pd.read_csv("data/ETHUSDT.csv")
BTCUSDT = pd.read_csv("data/BTCUSDT.csv")
ETHBTC = pd.read_csv("data/ETHBTC.csv")

In [3]:
# Convert the 'open_time' column from Unix timestamp (in milliseconds) to datetime
ETHUSDT['date_open'] = pd.to_datetime(ETHUSDT['open_time'], unit='ms')
ETHUSDT['date_close'] = pd.to_datetime(ETHUSDT['close_time'], unit='ms')
BTCUSDT['date_open'] = pd.to_datetime(BTCUSDT['open_time'], unit='ms')
BTCUSDT['date_close'] = pd.to_datetime(BTCUSDT['close_time'], unit='ms')
ETHBTC['date_open'] = pd.to_datetime(ETHBTC['open_time'], unit='ms')
ETHBTC['date_close'] = pd.to_datetime(ETHBTC['close_time'], unit='ms')

In [4]:
# Calculate the difference between date_close and date_open
ETHUSDT['time_difference_seconds'] = (ETHUSDT['date_close'] - ETHUSDT['date_open']).dt.total_seconds()
ETHUSDT['time_difference_minutes'] = round((ETHUSDT['time_difference_seconds']/60).astype(float))

BTCUSDT['time_difference_seconds'] = (BTCUSDT['date_close'] - BTCUSDT['date_open']).dt.total_seconds()
BTCUSDT['time_difference_minutes'] = round((BTCUSDT['time_difference_seconds']/60).astype(float))

ETHBTC['time_difference_seconds'] = (ETHBTC['date_close'] - ETHBTC['date_open']).dt.total_seconds()
ETHBTC['time_difference_minutes'] = round((ETHBTC['time_difference_seconds']/60).astype(float))

In [5]:
ETHUSDT1 = ETHUSDT[["date_open", "date_close", "close", "time_difference_minutes"]]
BTCUSDT1 = BTCUSDT[["date_open", "date_close", "close", "time_difference_minutes"]]
ETHBTC1 = ETHBTC[["date_open", "date_close", "close", "time_difference_minutes"]]

In [6]:
ETHUSDT1

,date_open,date_close,close,time_difference_minutes
0,2017-08-17 04:00:00,2017-08-17 04:14:59.999000064,298.00,15.0
1,2017-08-17 04:15:00,2017-08-17 04:29:59.999000064,299.39,15.0
2,2017-08-17 04:30:00,2017-08-17 04:44:59.999000064,299.60,15.0
3,2017-08-17 04:45:00,2017-08-17 04:59:59.999000064,301.61,15.0
4,2017-08-17 05:00:00,2017-08-17 05:14:59.999000064,302.01,15.0
...,...,...,...,...
207521,2023-07-24 17:30:00,2023-07-24 17:44:59.999000064,1842.48,15.0
207522,2023-07-24 17:45:00,2023-07-24 17:59:59.999000064,1843.30,15.0
207523,2023-07-24 18:00:00,2023-07-24 18:14:59.999000064,1845.00,15.0
207524,2023-07-24 18:15:00,2023-07-24 18:29:59.999000064,1847.47,15.0


In [7]:
# Calculate the log returns
ETHUSDT1['log_return'] = np.log(ETHUSDT1['close']/ETHUSDT1['close'].shift(1))
BTCUSDT1['log_return'] = np.log(BTCUSDT1['close']/BTCUSDT1['close'].shift(1))
ETHBTC1['log_return'] = np.log(ETHBTC1['close']/ETHBTC1['close'].shift(1))

C:\Users\oghen\AppData\Local\Temp/ipykernel_3684/2077641640.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ETHUSDT1['log_return'] = np.log(ETHUSDT1['close']/ETHUSDT1['close'].shift(1))
C:\Users\oghen\AppData\Local\Temp/ipykernel_3684/2077641640.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  BTCUSDT1['log_return'] = np.log(BTCUSDT1['close']/BTCUSDT1['close'].shift(1))
C:\Users\oghen\AppData\Local\Temp/ipykernel_3684/2077641640.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of 

In [8]:
ETHBTC1

,date_open,date_close,close,time_difference_minutes,log_return
0,2017-07-14 04:00:00,2017-07-14 04:14:59.999,0.086400,15.0,NaN
1,2017-07-14 04:15:00,2017-07-14 04:29:59.999,0.085811,15.0,-0.006840
2,2017-07-14 04:30:00,2017-07-14 04:44:59.999,0.086314,15.0,0.005845
3,2017-07-14 04:45:00,2017-07-14 04:59:59.999,0.086347,15.0,0.000382
4,2017-07-14 05:00:00,2017-07-14 05:14:59.999,0.084680,15.0,-0.019495
...,...,...,...,...,...
210787,2023-07-24 18:00:00,2023-07-24 18:14:59.999,0.063520,15.0,0.000472
210788,2023-07-24 18:15:00,2023-07-24 18:29:59.999,0.063570,15.0,0.000787
210789,2023-07-24 18:30:00,2023-07-24 18:44:59.999,0.063550,15.0,-0.000315
210790,2023-07-24 18:45:00,2023-07-24 18:59:59.999,0.063510,15.0,-0.000630


In [9]:
# Calculate the difference between consecutive date_open values in hours
ETHUSDT1['consecutive_time'] = ETHUSDT1['date_open'].diff().dt.total_seconds()
ETHUSDT1.dropna(subset=['consecutive_time'], inplace=True) # Drop rows with NaN in 'consecutive_time'
ETHUSDT1['consecutive_time_minutes'] = round((ETHUSDT1['consecutive_time']/60).astype(float))

BTCUSDT1['consecutive_time'] = BTCUSDT1['date_open'].diff().dt.total_seconds()
BTCUSDT1.dropna(subset=['consecutive_time'], inplace=True) # Drop rows with NaN in 'consecutive_time'
BTCUSDT1['consecutive_time_minutes'] = round((BTCUSDT1['consecutive_time']/60).astype(float))

ETHBTC1['consecutive_time'] = ETHBTC1['date_open'].diff().dt.total_seconds()
ETHBTC1.dropna(subset=['consecutive_time'], inplace=True) # Drop rows with NaN in 'consecutive_time'
ETHBTC1['consecutive_time_minutes'] = round((ETHBTC1['consecutive_time']/60).astype(float))

C:\Users\oghen\AppData\Local\Temp/ipykernel_3684/1342660602.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ETHUSDT1['consecutive_time'] = ETHUSDT1['date_open'].diff().dt.total_seconds()
C:\Users\oghen\AppData\Local\Temp/ipykernel_3684/1342660602.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ETHUSDT1.dropna(subset=['consecutive_time'], inplace=True) # Drop rows with NaN in 'consecutive_time'
C:\Users\oghen\AppData\Local\Temp/ipykernel_3684/1342660602.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try us

In [10]:
ETHUSDT1

,date_open,date_close,close,time_difference_minutes,log_return,consecutive_time,consecutive_time_minutes
1,2017-08-17 04:15:00,2017-08-17 04:29:59.999000064,299.39,15.0,0.004654,900.0,15.0
2,2017-08-17 04:30:00,2017-08-17 04:44:59.999000064,299.60,15.0,0.000701,900.0,15.0
3,2017-08-17 04:45:00,2017-08-17 04:59:59.999000064,301.61,15.0,0.006687,900.0,15.0
4,2017-08-17 05:00:00,2017-08-17 05:14:59.999000064,302.01,15.0,0.001325,900.0,15.0
5,2017-08-17 05:15:00,2017-08-17 05:29:59.999000064,302.21,15.0,0.000662,900.0,15.0
...,...,...,...,...,...,...,...
207521,2023-07-24 17:30:00,2023-07-24 17:44:59.999000064,1842.48,15.0,-0.000738,900.0,15.0
207522,2023-07-24 17:45:00,2023-07-24 17:59:59.999000064,1843.30,15.0,0.000445,900.0,15.0
207523,2023-07-24 18:00:00,2023-07-24 18:14:59.999000064,1845.00,15.0,0.000922,900.0,15.0
207524,2023-07-24 18:15:00,2023-07-24 18:29:59.999000064,1847.47,15.0,0.001338,900.0,15.0


In [11]:

#filtered_rows = 
#ETHUSDT1[ETHUSDT1['consecutive_time_minutes'] > 900]
#print(filtered_rows)
ETHBTC1[ETHBTC1['consecutive_time_minutes'] > 15]

,date_open,date_close,close,time_difference_minutes,log_return,consecutive_time,consecutive_time_minutes
5233,2017-09-06 23:00:00,2017-09-06 23:14:59.999,0.071986,15.0,-0.001638,25200.0,420.0
13710,2017-12-04 06:45:00,2017-12-04 06:59:59.999,0.040618,15.0,0.001700,2700.0,45.0
15077,2017-12-18 13:30:00,2017-12-18 13:44:59.999,0.038339,15.0,-0.001355,4500.0,75.0
16668,2018-01-04 05:00:00,2018-01-04 05:14:59.999,0.062376,15.0,0.014940,7200.0,120.0
20010,2018-02-09 10:00:00,2018-02-09 10:14:59.999,0.100299,15.0,0.000010,121500.0,2025.0
20090,2018-02-10 06:15:00,2018-02-10 06:29:59.999,0.100669,15.0,-0.001598,1800.0,30.0
20178,2018-02-11 04:30:00,2018-02-11 04:44:59.999,0.099906,15.0,-0.001290,1800.0,30.0
33128,2018-06-26 12:00:00,2018-06-26 12:14:59.999,0.072449,15.0,-0.011459,36900.0,615.0
33228,2018-06-27 14:45:00,2018-06-27 14:59:59.999,0.070200,15.0,-0.011120,7200.0,120.0
33843,2018-07-04 08:00:00,2018-07-04 08:14:59.999,0.071059,15.0,0.000127,27900.0,465.0


In [12]:
ETHBTC1[ETHBTC1['time_difference_minutes'] > 15]

,date_open,date_close,close,time_difference_minutes,log_return,consecutive_time,consecutive_time_minutes


In [14]:
ETHBTC1.head(2)

,date_open,date_close,close,time_difference_minutes,log_return,consecutive_time,consecutive_time_minutes
1,2017-07-14 04:15:00,2017-07-14 04:29:59.999,0.085811,15.0,-0.006840,900.0,15.0
2,2017-07-14 04:30:00,2017-07-14 04:44:59.999,0.086314,15.0,0.005845,900.0,15.0


In [15]:
# Rename columns to include prefixes
ETHBTC1.columns = ['ETHBTC1' + col if col != 'date_open' else col for col in ETHBTC1.columns]
ETHUSDT1.columns = ['ETHUSDT1' + col if col != 'date_open' else col for col in ETHUSDT1.columns]
BTCUSDT1.columns = ['BTCUSDT1' + col if col != 'date_open' else col for col in BTCUSDT1.columns]

# Merge df1 and df2 on 'date_open'
merged_df = pd.merge(BTCUSDT1, ETHUSDT1, on='date_open', how='inner')

# Merge the result with df3 on 'date_open'
merged_df = pd.merge(merged_df, ETHBTC1, on='date_open', how='inner')

# Now merged_df contains all the columns with the specified prefixes, and the rows are aligned based on 'date_open'

In [21]:
merged_df1 = merged_df[["BTCUSDT1log_return", "ETHUSDT1log_return", "ETHBTC1log_return"]]

In [33]:
# Loop through the columns and fit an AR model to each
for col in merged_df1:
    # Drop any NaN values
    series = merged_df1.BTCUSDT1log_return.dropna()

In [34]:
best_lag

97

In [35]:
best_aic = np.inf
best_lag = 0

# Define the maximum number of lags you want to consider
max_lags = 100

for lag in range(1, max_lags+1):
    model = AutoReg(series, lags=lag)
    result = model.fit()
    if result.aic < best_aic:
        best_aic = result.aic
        best_lag = lag
print("the best lag is ", best_lag)
# Fit the model with the best lag
model = AutoReg(series, lags=best_lag)
result = model.fit()
print(result.summary())

C:\Users\oghen\anaconda3\lib\site-packages\statsmodels\tsa\deterministic.py:1451: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(self._index, pd.Int64Index):
C:\Users\oghen\anaconda3\lib\site-packages\statsmodels\tsa\ar_model.py:248: FutureWarning: The parameter names will change after 0.12 is released. Set old_names to False to use the new names now. Set old_names to True to use the old names. 
  warnings.warn(
C:\Users\oghen\anaconda3\lib\site-packages\statsmodels\tsa\deterministic.py:1451: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(self._index, pd.Int64Index):
C:\Users\oghen\anaconda3\lib\site-packages\statsmodels\tsa\ar_model.py:248: FutureWarning: The parameter names will change after 0.12 is released. Set old_names to False to use t

the best lag is  100


C:\Users\oghen\anaconda3\lib\site-packages\statsmodels\tsa\deterministic.py:1451: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(self._index, pd.Int64Index):
C:\Users\oghen\anaconda3\lib\site-packages\statsmodels\tsa\ar_model.py:248: FutureWarning: The parameter names will change after 0.12 is released. Set old_names to False to use the new names now. Set old_names to True to use the old names. 
  warnings.warn(


                            AutoReg Model Results                             
Dep. Variable:     BTCUSDT1log_return   No. Observations:               207525
Model:                   AutoReg(100)   Log Likelihood              826191.198
Method:               Conditional MLE   S.D. of innovations              0.005
Date:                Tue, 15 Aug 2023   AIC                            -10.803
Time:                        10:05:01   BIC                            -10.798
Sample:                           100   HQIC                           -10.802
                               207525                                         
                              coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------
intercept                 1.01e-05    9.9e-06      1.020      0.308   -9.31e-06    2.95e-05
BTCUSDT1log_return.L1      -0.0189      0.002     -8.606      0.000      -0.023      -0.015


In [36]:
# Fit the model with the best lag (e.g., best_lag from previous code)
model = AutoReg(series, lags=best_lag)
result = model.fit()

# Forecast the next 230 values
forecast_values = result.predict(start=len(series), end=len(series) + 229, dynamic=True)

# The `dynamic` parameter ensures that the forecasted values are used for predictions once the lags go beyond the end of the sample data.

C:\Users\oghen\anaconda3\lib\site-packages\statsmodels\tsa\deterministic.py:1451: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(self._index, pd.Int64Index):
C:\Users\oghen\anaconda3\lib\site-packages\statsmodels\tsa\ar_model.py:248: FutureWarning: The parameter names will change after 0.12 is released. Set old_names to False to use the new names now. Set old_names to True to use the old names. 
  warnings.warn(
C:\Users\oghen\anaconda3\lib\site-packages\statsmodels\tsa\deterministic.py:141: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(index, pd.Int64Index) and np.all(np.diff(index) == 1):
C:\Users\oghen\anaconda3\lib\site-packages\statsmodels\tsa\deterministic.py:143: FutureWarning: pandas.Int64Index is deprecated and will be removed from 

In [37]:
forecast_values

207525    0.000010
207526   -0.000063
207527   -0.000104
207528   -0.000084
207529   -0.000099
            ...   
207750    0.000009
207751    0.000009
207752    0.000008
207753    0.000009
207754    0.000009
Length: 230, dtype: float64

In [38]:
series = merged_df1.ETHUSDT1log_return.dropna()

In [39]:
best_aic = np.inf
best_lag = 0

# Define the maximum number of lags you want to consider
max_lags = 100

for lag in range(1, max_lags+1):
    model = AutoReg(series, lags=lag)
    result = model.fit()
    if result.aic < best_aic:
        best_aic = result.aic
        best_lag = lag
print("the best lag is ", best_lag)

C:\Users\oghen\anaconda3\lib\site-packages\statsmodels\tsa\deterministic.py:1451: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(self._index, pd.Int64Index):
C:\Users\oghen\anaconda3\lib\site-packages\statsmodels\tsa\ar_model.py:248: FutureWarning: The parameter names will change after 0.12 is released. Set old_names to False to use the new names now. Set old_names to True to use the old names. 
  warnings.warn(
C:\Users\oghen\anaconda3\lib\site-packages\statsmodels\tsa\deterministic.py:1451: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(self._index, pd.Int64Index):
C:\Users\oghen\anaconda3\lib\site-packages\statsmodels\tsa\ar_model.py:248: FutureWarning: The parameter names will change after 0.12 is released. Set old_names to False to use t

the best lag is  100


In [40]:
# Fit the model with the best lag
model1 = AutoReg(series, lags=best_lag)
result = model1.fit()
print(result.summary())

C:\Users\oghen\anaconda3\lib\site-packages\statsmodels\tsa\deterministic.py:1451: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(self._index, pd.Int64Index):
C:\Users\oghen\anaconda3\lib\site-packages\statsmodels\tsa\ar_model.py:248: FutureWarning: The parameter names will change after 0.12 is released. Set old_names to False to use the new names now. Set old_names to True to use the old names. 
  warnings.warn(


                            AutoReg Model Results                             
Dep. Variable:     ETHUSDT1log_return   No. Observations:               207525
Model:                   AutoReg(100)   Log Likelihood              785916.393
Method:               Conditional MLE   S.D. of innovations              0.005
Date:                Tue, 15 Aug 2023   AIC                            -10.415
Time:                        10:54:57   BIC                            -10.410
Sample:                           100   HQIC                           -10.413
                               207525                                         
                              coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------
intercept                9.413e-06    1.2e-05      0.783      0.434   -1.41e-05     3.3e-05
ETHUSDT1log_return.L1      -0.0049      0.002     -2.254      0.024      -0.009      -0.001


In [41]:
series = merged_df1.ETHBTC1log_return.dropna()

In [42]:
# Fit the model with the best lag (e.g., best_lag from previous code)
model2 = AutoReg(series, lags=best_lag)
result = model2.fit()

# Forecast the next 230 values
forecast_values = result.predict(start=len(series), end=len(series) + 229, dynamic=True)

C:\Users\oghen\anaconda3\lib\site-packages\statsmodels\tsa\deterministic.py:1451: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(self._index, pd.Int64Index):
C:\Users\oghen\anaconda3\lib\site-packages\statsmodels\tsa\ar_model.py:248: FutureWarning: The parameter names will change after 0.12 is released. Set old_names to False to use the new names now. Set old_names to True to use the old names. 
  warnings.warn(
C:\Users\oghen\anaconda3\lib\site-packages\statsmodels\tsa\deterministic.py:141: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(index, pd.Int64Index) and np.all(np.diff(index) == 1):
C:\Users\oghen\anaconda3\lib\site-packages\statsmodels\tsa\deterministic.py:143: FutureWarning: pandas.Int64Index is deprecated and will be removed from 

In [ ]:
# Fit the model with the best lag (e.g., best_lag from previous code)
model = AutoReg(series, lags=best_lag)
result = model.fit()

# Forecast the next 230 values
forecast_values = result.predict(start=len(series), end=len(series) + 229, dynamic=True)

# The `dynamic` parameter ensures that the forecasted values are used for predictions once the lags go beyond the end of the sample data.

In [ ]:
# Fit the model with the best lag
model = AutoReg(series, lags=best_lag)
result = model.fit()
print(result.summary())